# The Battle of Neighborhoods (Week 1)

# Finding a Better Place in Washington DC to open Ethiopian cultural restaurant.

## Introduction
### Background
As per the information from Ethiopian Embassy in united state there are more than 200,000 Ethiopian immigrant are living in Washington DC. Compared to other states of U.S majority of the Ethiopian immigrant are living in Washington DC.
This project aims to estimate the best localization to open such a Ethiopian Cultural restaurant in the city of Washington DC.
Prior to starting any restaurant, it’s very crucial to know the convenient location where to launch the restaurant. In order to do so, this project will try to gather data about existing Ethiopian restaurant localization in Washington city neighborhood.

### Problem 
As the goal is to find the best place to open Ethiopian restaurant, we need to make sure that the place should be a locality for Ethiopian. We also need to check that customer could be interested in this specific business. In order to do so, a survey in Washington neighborhood will be done in addition to data gathering. What kinds of restaurant works well? This survey will allow to validate the data analysis done here.

## Data 
Based on definition of our problem, factors that will influence our decision are:
• Finding the best place where Ethiopian restaurant are mostly found.
• finding the most common venues
• choosing the right neighborhood within the borough
We will be using the geographical coordinates of Washington DC to plot neighborhoods in a borough that is most popular by Ethiopian immigrant, and finally cluster our neighborhoods and present our findings.


# Methodology

# Analysis 

#### 1. Displaying city neighbourhoods

Before we get the data and start exploring it, let's download all the dependencies that we will need 

#### Importing the necessary module 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install geocoder
#!conda install -c conda-forge geopy --yes
import geopy
import geocoder # to get coordinates

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011
Libraries imported.


#### Download and Explore Dataset
 Geodata of Washington can be found https://query.data.world/s/i3n2uliq7hfcumldfarhh2rkcxlbe5

In [5]:

wdc = pd.read_csv('https://query.data.world/s/i3n2uliq7hfcumldfarhh2rkcxlbe5')  

In [6]:
wdc.shape

(39, 416)

In [7]:
wdc.head()

,neighborhood_cluster,Cluster_num,Short_Name,Full_Name,lon_ctr,lat_ctr,Population,"Population,_1980","Population,_1990","Population,_2000","Population,_2010","Pct_chng_pop,_1980_to_1990","Pct_chng_pop,_1990_to_1900","Pct_chng_pop,_2000_to_2010",Children,"Pct_children,_1980","Pct_children,_1990","Pct_children,_2000","Pct_children,_2010","Pct_chng_child_pop,_1980_to_90","Pct_chng_child_pop,_1990_to_00","Pct_chng_child_pop,_2000_to_10",Race/Ethnicity,"Pct_black_non-Hispanic,_1990","Pct_black_non-Hispanic,_2000","Pct_black_non-Hispanic,_2010","Pct_white_non-Hispanic,_1990","Pct_white_non-Hispanic,_2000","Pct_white_non-Hispanic,_2010","Pct_Hispanic,_1990","Pct_Hispanic,_2000","Pct_Hispanic,_2010",Pct_Asian/PI_non-Hispanic_1990,Pct_Asian/PI_non-Hispanic_2000,Pct_Asian/PI_non-Hispanic_2010,Foreign-Born,"Pct_foreign_born,_1980","Pct_foreign_born,_1990","Pct_foreign_born,_2000","Pct_foreign_born,_2007-11",Families,Pct_female-head_fams_w_chld_90,Pct_female-head_fams_w_chld_00,Pct_feml-head_fam_w_chld_07-11,Low_weight_births,Pct_low_weight_births_1998,Pct_low_weight_births_1999,Pct_low_weight_births_2000,Pct_low_weight_births_2001,Pct_low_weight_births_2002,Pct_low_weight_births_2003,Pct_low_weight_births_2004,Pct_low_weight_births_2005,Pct_low_weight_births_2006,Pct_low_weight_births_2007,Pct_low_weight_births_2008,Pct_low_weight_births_2009,Pct_low_weight_births_2010,Pct_low_weight_births_2011,Births_to_teen_mothers,Pct_birth_to_teen_mothers_1998,Pct_birth_to_teen_mothers_1999,Pct_birth_to_teen_mothers_2000,Pct_birth_to_teen_mothers_2001,Pct_birth_to_teen_mothers_2002,Pct_birth_to_teen_mothers_2003,Pct_birth_to_teen_mothers_2004,Pct_birth_to_teen_mothers_2005,Pct_birth_to_teen_mothers_2006,Pct_birth_to_teen_mothers_2007,Pct_birth_to_teen_mothers_2008,Pct_birth_to_teen_mothers_2009,Pct_birth_to_teen_mothers_2010,Pct_birth_to_teen_mothers_2011,Poverty,"Poverty_rate_(Pct),_1980","Poverty_rate_(Pct),_1990","Poverty_rate_(Pct),_2000","Poverty_rate_(Pct),_2007-11","Pct_children_in_poverty,_1990","Pct_children_in_poverty,_2000",Pct_child_in_poverty_2007-11,Employment,"Unemployment_rate_(Pct),_1980","Unemployment_rate_(Pct),_1990","Unemployment_rate_(Pct),_2000","Unemployment_rate,_2007-11",Pct_pop_16+_yrs_employed_1980,Pct_pop_16+_yrs_employed_1990,Pct_pop_16+_yrs_employed_2000,Pct_pop_16+_yrs_employed_07-11,Education,Pct_persons_wo_HS_diploma_1980,Pct_persons_wo_HS_diploma_1990,Pct_persons_wo_HS_diploma_2000,Pct_persons_wo_HS_dip_2007-11,Isolation,"Pct_HHs_with_a_phone,_2000","Pct_HHs_with_a_phone,_2007-11","Pct_HHs_with_a_car,_2000","Pct_HHs_with_a_car,_2007-11",Family_income_(2010_$),"Avg._family_income,_1979","Avg._family_income,_1989","Avg._family_income,_1999","Avg._family_income,_2007-11",Pct_chng_avg_fam_inc_1980-90,Pct_chng_avg_fam_inc_1990-00,Pct_chng_avg_fam_inc_90to07-11,Food_stamps,Person_receiv_food_stamps_2000,Person_receiv_food_stamps_2001,Person_receiv_food_stamps_2002,Person_receiv_food_stamps_2003,Person_receiv_food_stamps_2004,Person_receiv_food_stamps_2005,Person_receiv_food_stamps_2006,Person_receiv_food_stamps_2007,Person_receiv_food_stamps_2008,Person_receiv_food_stamps_2009,Person_receiv_food_stamps_2010,Person_receiv_food_stamps_2011,Person_receiv_food_stamps_2012,Person_receiv_food_stamps_2013,TANF,"Persons_receiving_TANF,_2000","Persons_receiving_TANF,_2001","Persons_receiving_TANF,_2002","Persons_receiving_TANF,_2003","Persons_receiving_TANF,_2004","Persons_receiving_TANF,_2005","Persons_receiving_TANF,_2006","Persons_receiving_TANF,_2007","Persons_receiving_TANF,_2008","Persons_receiving_TANF,_2009","Persons_receiving_TANF,_2010","Persons_receiving_TANF,_2011","Persons_receiving_TANF,_2012","Persons_receiving_TANF,_2013","Violent_Crimes_(per_1,000_pop)","Violent_crimes,_2000","Violent_crimes,_2001","Violent_crimes,_2002","Violent_crimes,_2003","Violent_crimes,_2004","Violent_crimes,_2005","Violent_crimes,_2006","Violent_crimes,_2007","Violent_crimes,_2008","Violent_crimes,_2009","Violent_crimes,_2010","Violent_cr

Filter the relevant data from the main dataset 

In [8]:
# define the dataframe columns
neighborhoods =wdc[['neighborhood_cluster', 'Cluster_num','Short_Name', 'lat_ctr', 'lon_ctr']]

In [9]:
neighborhoods.shape

(39, 5)

In [10]:
print('The dataframe has {} cluster and {} neighborhoods.'.format(
        len(neighborhoods['Cluster_num'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 39 cluster and 39 neighborhoods.


Let's explore the coordinate of Washington city

In [12]:
address = 'Washington, DC'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Washington City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Washington City are 38.8949924, -77.0365581.


#### Create a map of Washington DC where neihborhood super imposed 

In [13]:
# create map of Washington using latitude and longitude values
map_washington = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['lat_ctr'], neighborhoods['lon_ctr'], neighborhoods['neighborhood_cluster'], neighborhoods['Short_Name']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington)  
    
map_washington

Let's get the geographical coordinates of Adams Morgan, Brightwood, and Columbia Heights and shaw where most of Ethiopian immigrant lives.

In [14]:
address = 'Adams Morgan, DC'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
AM_latitude = location.latitude
AM_longitude = location.longitude
print('The geograpical coordinate of Adams Morgan are {}, {}.'.format(AM_latitude, AM_longitude))

The geograpical coordinate of Adams Morgan are 38.9215002, -77.0421992.


In [15]:
CLIENT_ID = 'XXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P51Z1V2PLFCRVTIYIR4YHHZ4OFZDA4VEZKWIVEEHMYY41GR2
CLIENT_SECRET:YGMUNTUCJJLAZAEKNVJ1U5NY2KFEAVQ3CPOFENKOUCIVS5VA


#### Now, let's get the top venues in Adams Morgan within a radius of 500 meters.

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    AM_latitude, 
    AM_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=P51Z1V2PLFCRVTIYIR4YHHZ4OFZDA4VEZKWIVEEHMYY41GR2&client_secret=YGMUNTUCJJLAZAEKNVJ1U5NY2KFEAVQ3CPOFENKOUCIVS5VA&v=20180605&ll=38.9215002,-77.0421992&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5feeeec45c2c4d623fe68d69'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Adams Morgan',
  'headerFullLocation': 'Adams Morgan, Washington',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 38.9260002045, 'lng': -77.03642598732625},
   'sw': {'lat': 38.917000195499995, 'lng': -77.04797241267374}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55390c7c498e5c63a1376fb4',
       'name': 'Lapis',
       'location': {'address': '1847 Columbia Rd NW',
        'crossStreet': 'at Mintwood Pl NW',
        'lat': 38.92130182752878,
        'lng': -77.04388976097107,
        'labeledLatLngs': [{'label

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lapis,Afghan Restaurant,38.921302,-77.043890
1,The LINE Hotel,Hotel,38.922815,-77.041688
2,The Green Zone,Cocktail Bar,38.919483,-77.041845
3,Donburi,Japanese Restaurant,38.921673,-77.042385
4,Smoke & Barrel,BBQ Joint,38.922266,-77.042286


And how many venues were returned by Foursquare?

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


## Let's create a function to repeat the same process to all the neighborhoods in Washington DC

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
washington_venues = getNearbyVenues(names=wdc['Short_Name'],
                                   latitudes=wdc['lat_ctr'],
                                   longitudes=wdc['lon_ctr']
                                  )

Adams Morgan
Columbia Heights
Shaw
Georgetown
Foggy Bottom
DuPont Circle
Logan Circle
Downtown
Southwest
Chevy Chase
Friendship Heights
Van Ness
Palisades
Glover Park
Cleveland Park
Shepherd Park
Takoma
Petworth
Lamond Riggs
Michigan Park
Edgewood
Brookland
Ivy City
Woodridge
Kingman Park
South Capitol Hill
Navy Yard
Anacostia
Kenilworth
Mayfair
Deanwood
Fort DuPont
Capitol View
Fairlawn
Naylor Gardens
Garfield Heights
Barry Farm
Shipley Terrace
Bellevue


In [23]:
print(washington_venues.shape)
washington_venues.head()

(817, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams Morgan,38.919379,-77.047702,Sushi Ogawa,38.917900,-77.047768,Sushi Restaurant
1,Adams Morgan,38.919379,-77.047702,Lapis,38.921302,-77.043890,Afghan Restaurant
2,Adams Morgan,38.919379,-77.047702,Lapop,38.921302,-77.043777,Lounge
3,Adams Morgan,38.919379,-77.047702,Pitango Gelato & Café,38.921412,-77.043801,Ice Cream Shop
4,Adams Morgan,38.919379,-77.047702,So's Your Mom,38.921671,-77.043753,Bagel Shop


Let's check how many venues were returned for each neighborhood

In [24]:
washington_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adams Morgan,29,29,29,29,29,29
Anacostia,6,6,6,6,6,6
Barry Farm,8,8,8,8,8,8
Bellevue,4,4,4,4,4,4
Brookland,9,9,9,9,9,9
Capitol View,4,4,4,4,4,4
Chevy Chase,4,4,4,4,4,4
Cleveland Park,5,5,5,5,5,5
Columbia Heights,56,56,56,56,56,56


Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(washington_venues['Venue Category'].unique())))

There are 219 uniques categories.


## Analyze Each Neighborhood

Let's create one hot encoding 

In [26]:
# one hot encoding
washington_onehot = pd.get_dummies(washington_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
washington_onehot['Neighborhood'] = washington_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [washington_onehot.columns[-1]] + list(washington_onehot.columns[:-1])
washington_onehot = washington_onehot[fixed_columns]

washington_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chiropractor,Chocolate Shop,Christmas Market,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Library,College Quad,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lawyer,Liquor Store,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Optical Shop,Outdoor Sculpture,Pakistani Restaurant,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Swiss Restaurant,Synagogue,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adams Morgan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adams Morgan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
washington_grouped = washington_onehot.groupby('Neighborhood').mean().reset_index()
washington_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chiropractor,Chocolate Shop,Christmas Market,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Library,College Quad,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lawyer,Liquor Store,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Optical Shop,Outdoor Sculpture,Pakistani Restaurant,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Swiss Restaurant,Synagogue,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adams Morgan,0.0,0.034483,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.000,0.0,0.00,0.00,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.034483,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068966,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.068966,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.034483,0.0,0.103448,0.0,0.034483,0.0,0.0,0.0,0.000000,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.034483,0.0,0.0,0.0,0.034483,0.000000,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.000,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.034483,0.0,0.0,0.0,0.0,0.0

#### Let's confirm the new size

In [30]:
washington_grouped.shape

(39, 219)

### Let's print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in washington_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = washington_grouped[washington_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adams Morgan----
                  venue  freq
0                 Hotel  0.10
1               Dog Run  0.07
2  Gym / Fitness Center  0.07
3      Sushi Restaurant  0.07
4           Coffee Shop  0.03


----Anacostia----
                  venue  freq
0        History Museum  0.17
1  Fast Food Restaurant  0.17
2        Sandwich Place  0.17
3              Boutique  0.17
4     Convenience Store  0.17


----Barry Farm----
               venue  freq
0     History Museum  0.12
1                Gym  0.12
2  Convenience Store  0.12
3                Spa  0.12
4  Outdoor Sculpture  0.12


----Bellevue----
              venue  freq
0    Baseball Field  0.25
1      Costume Shop  0.25
2              Park  0.25
3  Basketball Court  0.25
4      Optical Shop  0.00


----Brookland----
           venue  freq
0   Liquor Store  0.22
1        Dog Run  0.11
2   Intersection  0.11
3     Skate Park  0.11
4  Memorial Site  0.11


----Capitol View----
                venue  freq
0        Soccer Field  0.25
1   

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = washington_grouped['Neighborhood']

for ind in np.arange(washington_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(washington_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams Morgan,Hotel,Sushi Restaurant,Gym / Fitness Center,Dog Run,Lounge,Ice Cream Shop,Sports Bar,Italian Restaurant,Shipping Store,Seafood Restaurant
1,Anacostia,Fast Food Restaurant,History Museum,American Restaurant,Convenience Store,Sandwich Place,Boutique,Dive Bar,Farmers Market,Falafel Restaurant,Event Space
2,Barry Farm,History Museum,Convenience Store,Coffee Shop,Gym,Chinese Restaurant,Bank,Spa,Outdoor Sculpture,Women's Store,Donut Shop
3,Bellevue,Baseball Field,Costume Shop,Park,Basketball Court,Women's Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dry Cleaner
4,Brookland,Liquor Store,Memorial Site,Skate Park,Brewery,Drugstore,Gas Station,Dog Run,Intersection,Discount Store,Falafel Restaurant


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [34]:
# set number of clusters
kclusters = 5

washington_grouped_clustering = washington_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(washington_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# add clustering labels
wdc_new=wdc[['Short_Name', 'lat_ctr', 'lon_ctr']]
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

washington_merged = wdc_new

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
washington_merged = washington_merged.join(neighborhoods_venues_sorted)

washington_merged.head() # check the last columns!

,Short_Name,lat_ctr,lon_ctr,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams Morgan,38.919379,-77.047702,0,Adams Morgan,Hotel,Sushi Restaurant,Gym / Fitness Center,Dog Run,Lounge,Ice Cream Shop,Sports Bar,Italian Restaurant,Shipping Store,Seafood Restaurant
1,Columbia Heights,38.928700,-77.031769,0,Anacostia,Fast Food Restaurant,History Museum,American Restaurant,Convenience Store,Sandwich Place,Boutique,Dive Bar,Farmers Market,Falafel Restaurant,Event Space
2,Shaw,38.921257,-77.023415,0,Barry Farm,History Museum,Convenience Store,Coffee Shop,Gym,Chinese Restaurant,Bank,Spa,Outdoor Sculpture,Women's Store,Donut Shop
3,Georgetown,38.909686,-77.066409,0,Bellevue,Baseball Field,Costume Shop,Park,Basketball Court,Women's Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dry Cleaner
4,Foggy Bottom,38.902386,-77.048975,0,Brookland,Liquor Store,Memorial Site,Skate Park,Brewery,Drugstore,Gas Station,Dog Run,Intersection,Discount Store,Falafel Restaurant


Finally, let's visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(washington_merged['lat_ctr'], washington_merged['lon_ctr'], washington_merged['Short_Name'], washington_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Conclusion-  as per the analysis result from 39 neighborhoods Adams Morgan and Down Town are selected for launching Ethiopian cultural restuarant. The selection is based on the criteria that the place where most Ethiopian immigrants are residing and the neighborhoods should have a venue for other countries hotels and restuarant.